<a href="https://colab.research.google.com/github/njgeorge000158/deep-learning-challenge/blob/main/AlphabetSoupCharityOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [123]:
#*******************************************************************************************
 #
 #  File Name:  AlphabetSoupCharityOptimization.ipynb
 #
 #  File Description:
 #      This interactive Python notebook,AlphabetSoupCharityOptimization.ipynb, reads
 #      a csv file, LendingData.csv, in the Resources folder, and uses Python and neural
 #      networks methods to use the features in the provided dataset to create a binary
 #      classifier that can predict whether applicants for charity donations will be
 #      successful if funded by the nonprofit foundation, Alphabet Soup.
 #
 #      I reinstalled the scikit-learn module using the following commands to allow the
 #      RandomOverSampler function to work:
 #
 #      pip3 uninstall scikit-learn
 #      pip3 install scikit-learn==1.2.2
 #
 #
 #  Date            Description                             Programmer
 #  ----------      ------------------------------------    ------------------
 #  12/03/2023      Initial Development                     N. James George
 #
 #******************************************************************************************/

from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.insert(0,'/content/drive/My Drive/deep-learning-challenge')

!pip install -q dataframe_image
!pip install aspose-words
!pip install hvplot
!pip install silence-tensorflow
!pip install keras-tuner

import PyConstants as constant
import PyFunctions as function
import PyLogConstants as log_constant
import PyLogFunctions as log_function
import PyLogSubRoutines as log_subroutine
import PySubRoutines as subroutine

import AlphabetSoupCharityOptimizationSearchFunctions as local_function

import pandas as pd

import json

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [124]:
CONSTANT_LOCAL_FILE_NAME \
    = 'AlphabetSoupCharityOptimization.ipynb'

CONSTANT_CHARITY_DATA_CSV_FILE_PATH \
    = '/content/drive/My Drive/deep-learning-challenge/Resources/CharityData.csv'

CONSTANT_OPTIMIZATION_FILE_PATH \
    = '/content/drive/My Drive/deep-learning-challenge/Resources/AlphabetSoupCharityOptimization.keras'

log_subroutine \
    .SetLogMode \
        (False)

log_subroutine \
    .SetDebugMode \
        (False)

log_subroutine \
    .SetImageMode \
        (False)

log_subroutine \
    .BeginProgramExecution \
        ('AlphabetSoupCharityOptimization')

# <br> **Section 1: Data Acquistion**

## **1.1: Import and Read Charity Data**

In [125]:
charityDataFrame \
    = function. \
        ReturnCSVFileAsDataFrame \
            (CONSTANT_CHARITY_DATA_CSV_FILE_PATH)


log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

## **1.2: Display Charity Data**

In [126]:
captionString \
    = 'Table 1.2.1: Charity Data Table'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (charityDataFrame.head(12),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
"10,520,599",BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,"5,000",1
"10,531,628",AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,"108,590",1
"10,547,893",ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0
"10,553,066",SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,"6,692",1
"10,556,103",GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,"142,590",1
"10,556,855",MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,Independent,C1200,Preservation,Trust,1,0,N,"5,000",1
"10,558,440",FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,"31,452",1
"10,566,033",ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,"7,508,025",1
"10,570,430",ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,"94,389",1
"10,571,689",INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0


# <br> **Section 2: Preprocessing**

## **2.1: Drop the Non-Beneficial ID Columns**

In [127]:
charityDataFrame \
    .drop \
        (['EIN', 'STATUS', 'SPECIAL_CONSIDERATIONS'],
         axis = 1,
         inplace = True)

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

In [128]:
captionString \
    = 'Table 1.2.2: Modified Charity Data Table'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (charityDataFrame.head(12),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,"5,000",1
AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,"108,590",1
ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,"5,000",0
SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,"6,692",1
GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,"142,590",1
MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,Independent,C1200,Preservation,Trust,0,"5,000",1
FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,Independent,C1000,Preservation,Trust,100000-499999,"31,452",1
ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,10M-50M,"7,508,025",1
ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1-9999,"94,389",1
INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,"5,000",0


## **2.2: Determine the Number of Unique Values in Each Column**

In [129]:
numberOfUniqueValuesInEachColumnSeries \
    = charityDataFrame \
        .nunique()

log_function \
    .DebugReturnObjectWriteObject \
        (numberOfUniqueValuesInEachColumnSeries)

In [130]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(numberOfUniqueValuesInEachColumnSeries))

NAME                19568
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
ORGANIZATION            4
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64


## **2.3: NAME Binning**

### **Look at NAME Value Counts for Binning**

In [131]:
nameValueCountsForBinningSeries \
    = charityDataFrame['NAME'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

nameValueCountsForBinningSeries \
    .name = 'NAME'

log_function \
    .DebugReturnObjectWriteObject \
        (nameValueCountsForBinningSeries)

### **Choose a Cutoff Value and Create a List of Names to be Replaced**

In [132]:
nameCutoffInteger = 2

names_to_replace \
    = list \
        (nameValueCountsForBinningSeries \
            [nameValueCountsForBinningSeries < nameCutoffInteger] \
          .index)

log_function \
    .DebugReturnObjectWriteObject \
        (names_to_replace)

### **Replace Application Types in DataFrame**

In [133]:
for name in names_to_replace:

    charityDataFrame['NAME'] \
        = charityDataFrame['NAME'] \
            .replace \
                (name, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [134]:
nameValueCountsForBinningSeries \
    = charityDataFrame['NAME'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

nameValueCountsForBinningSeries \
    .name = 'NAME'

log_function \
    .DebugReturnObjectWriteObject \
        (nameValueCountsForBinningSeries)

## **2.3: APPLICATION_TYPE Binning**

### **Look at APPLICATION_TYPE Value Counts for Binning**

In [135]:
applicationTypeValueCountsForBinningSeries \
    = charityDataFrame['APPLICATION_TYPE'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

applicationTypeValueCountsForBinningSeries \
    .name = 'APPLICATION_TYPE'

log_function \
    .DebugReturnObjectWriteObject \
        (applicationTypeValueCountsForBinningSeries)

In [136]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(applicationTypeValueCountsForBinningSeries))

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


### **Choose a Cutoff Value and Create a List of Application Types to be Replaced**

In [137]:
applicationTypeCutoffInteger = 157

application_types_to_replace \
    = list \
        (applicationTypeValueCountsForBinningSeries \
             [applicationTypeValueCountsForBinningSeries < applicationTypeCutoffInteger] \
          .index)

log_function \
    .DebugReturnObjectWriteObject \
        (application_types_to_replace)

In [138]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(application_types_to_replace))

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


### **Replace Application Types in DataFrame**

In [139]:
for applicationTypes in application_types_to_replace:

    charityDataFrame['APPLICATION_TYPE'] \
        = charityDataFrame['APPLICATION_TYPE'] \
            .replace \
                (applicationTypes, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [140]:
applicationTypeValueCountsForBinningSeries \
    = charityDataFrame['APPLICATION_TYPE'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

applicationTypeValueCountsForBinningSeries \
    .name = 'APPLICATION_TYPE'

log_function \
    .DebugReturnObjectWriteObject \
        (applicationTypeValueCountsForBinningSeries)

In [141]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(applicationTypeValueCountsForBinningSeries))

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


## **2.4: CLASSIFICATION Binning**

### **Look at CLASSIFICATION Value Counts for Binning**

In [142]:
classificationValueCountsForBinningSeries \
    = charityDataFrame['CLASSIFICATION'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

classificationValueCountsForBinningSeries \
    .name = 'CLASSIFICATION'

log_function \
    .DebugReturnObjectWriteObject \
        (classificationValueCountsForBinningSeries)

In [143]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(classificationValueCountsForBinningSeries))

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C5200        1
C2600        1
C4200        1
C2190        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


### **Look at CLASSIFICATION Value Counts > 1**

In [144]:
classificationValueCountsForBinningSeries \
    [classificationValueCountsForBinningSeries > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1600        5
C1257        5
C1260        3
C0           3
C2710        3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

### **Choose a Cutoff Value and Create a List of Classifications to be Replaced**

In [145]:
classificationCutoffInteger = 96

classifications_to_replace \
    = list \
        (classificationValueCountsForBinningSeries \
             [classificationValueCountsForBinningSeries < classificationCutoffInteger] \
          .index)

log_function \
    .DebugReturnObjectWriteObject \
        (classifications_to_replace)

### **Replace Classifications in DataFrame**

In [146]:
for classification in classifications_to_replace:

    charityDataFrame['CLASSIFICATION'] \
        = charityDataFrame['CLASSIFICATION'] \
            .replace \
                (classification, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [147]:
classificationValueCountsForBinningSeries \
    = charityDataFrame['CLASSIFICATION'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

classificationValueCountsForBinningSeries \
    .name = 'CLASSIFICATION'

log_function \
    .DebugReturnObjectWriteObject \
        (classificationValueCountsForBinningSeries)

In [148]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(classificationValueCountsForBinningSeries))

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64


## **2.5: USE_CASE Binning**

### **Look at USE_CASE Value Counts for Binning**

In [149]:
useCaseValueCountsForBinningSeries \
    = charityDataFrame['USE_CASE'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

useCaseValueCountsForBinningSeries \
    .name = 'USE_CASE'

log_function \
    .DebugReturnObjectWriteObject \
        (useCaseValueCountsForBinningSeries)

In [150]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(useCaseValueCountsForBinningSeries))

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64


### **Choose a Cutoff Value and Create a List of Use Cases to be Replaced**

In [151]:
useCaseCutoffInteger = 147

use_cases_to_replace \
    = list \
        (useCaseValueCountsForBinningSeries \
             [useCaseValueCountsForBinningSeries < useCaseCutoffInteger] \
          .index)

log_function \
    .DebugReturnObjectWriteObject \
        (use_cases_to_replace)

In [152]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(use_cases_to_replace))

['Heathcare', 'Other']


### **Replace Use Cases in DataFrame**

In [153]:
for useCase in use_cases_to_replace:

    charityDataFrame['USE_CASE'] \
        = charityDataFrame['USE_CASE'] \
            .replace \
                (useCase, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [154]:
useCaseValueCountsForBinningSeries \
    = charityDataFrame['USE_CASE'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

useCaseValueCountsForBinningSeries \
    .name = 'USE_CASE'

log_function \
    .DebugReturnObjectWriteObject \
        (useCaseValueCountsForBinningSeries)

In [155]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(useCaseValueCountsForBinningSeries))

Preservation     28095
ProductDev        5671
CommunityServ      384
Other              149
Name: USE_CASE, dtype: int64


## **2.6: ASK_AMT Binning**

### **Look at ASK_AMT Value Counts for Binning**

In [156]:
askAmtValueCountsForBinningSeries \
    = charityDataFrame['ASK_AMT'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

askAmtValueCountsForBinningSeries \
    .name = 'ASK_AMT'

log_function \
    .DebugReturnObjectWriteObject \
        (askAmtValueCountsForBinningSeries)

In [157]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(askAmtValueCountsForBinningSeries))

5000        25398
15583           3
63981           3
6725            3
10478           3
            ...  
11336           1
1270766         1
121619          1
46915           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64


### **Choose a Cutoff Value and Create a List of Ask Amounts to be Replaced**

In [158]:
askAmtCutoffInteger = 4

ask_amounts_to_replace \
    = list \
        (askAmtValueCountsForBinningSeries \
             [askAmtValueCountsForBinningSeries < askAmtCutoffInteger] \
          .index)

log_function \
    .DebugReturnObjectWriteObject \
        (ask_amounts_to_replace)

In [159]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(ask_amounts_to_replace))

[15583, 63981, 6725, 10478, 34538, 73627, 32807, 16200, 12238, 64093, 41894, 57957, 57689, 9413, 34792, 15372, 61397, 1427468, 24105, 5130, 57588, 7049, 27944, 6405, 16071, 13499, 24459, 27441, 14000, 18985, 63520, 5179, 14765, 201112, 350000, 32507, 8746, 22472, 9323, 5548, 19704, 16435, 238933, 40743, 20684, 25952, 9885, 159223, 80168, 37489, 30769, 150599, 27159, 100000, 8181, 24746, 45793, 240780, 5894, 20537, 29541, 6010, 16482, 56135, 37390, 60000, 195661, 18826, 9255, 21346, 7287, 210432, 40429, 17735, 20071, 5255, 7187, 5006, 13898, 7686, 17614, 31453, 42962, 26432, 18063, 23202, 6960, 5297, 21389, 246851, 94253, 6000, 146824, 12753, 15039, 20834, 12584, 47129, 80599, 7003, 31549, 42072, 5731, 46327, 11839, 6742, 21264, 86039, 73987, 9537, 29130, 206725, 13420, 136341, 39168, 39122, 10070, 34995, 83211, 35734, 5170, 36683, 51366, 50890, 32141, 5175, 6920, 8632, 83827, 59647, 15722, 43814, 22579, 67367, 64471, 54024, 34875, 27073, 5706, 6279, 17010, 184668, 5470, 6398, 20150, 90

### **Replace Ask Amounts in DataFrame**

In [160]:
for askAmt in ask_amounts_to_replace:

    charityDataFrame['ASK_AMT'] \
        = charityDataFrame['ASK_AMT'] \
            .replace \
                (askAmt, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [161]:
askAmtValueCountsForBinningSeries \
    = charityDataFrame['ASK_AMT'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

askAmtValueCountsForBinningSeries \
    .name = 'ASK_AMT'

log_function \
    .DebugReturnObjectWriteObject \
        (askAmtValueCountsForBinningSeries)

In [162]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(askAmtValueCountsForBinningSeries))

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64


## **2.7: Final Preprocessing**

### **Convert Categorical Data to Numeric with `pd.get_dummies`**

In [163]:
dummiesDataFrame \
    = pd.get_dummies \
        (charityDataFrame)

log_function \
    .DebugReturnObjectWriteObject \
        (dummiesDataFrame)

In [164]:
captionString \
    = 'Table 2.5.1: Dummies DataFrame Table'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (dummiesDataFrame.head(12),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

### **Split Preprocessed Data into Features and Target Arrays**

In [165]:
yNumpyArray \
    = dummiesDataFrame['IS_SUCCESSFUL'].values

log_function \
    .DebugReturnObjectWriteObject \
        (yNumpyArray)

In [166]:
XNumpyArray \
    = dummiesDataFrame \
        .drop \
            (['IS_SUCCESSFUL'],
             axis = 1) \
        .values

log_function \
    .DebugReturnObjectWriteObject \
        (XNumpyArray)

### **Split the Preprocessed Data into a Training and Testing Dataset**

In [167]:
XTrainNumpyArray, \
XTestNumpyArray, \
yTrainNumpyArray, \
yTestNumpyArray \
    = train_test_split \
            (XNumpyArray,
             yNumpyArray,
             random_state = 2)

In [168]:
log_function \
    .DebugReturnObjectWriteObject \
        (XTrainNumpyArray)

In [169]:
log_function \
    .DebugReturnObjectWriteObject \
        (XTestNumpyArray)

In [170]:
log_function \
    .DebugReturnObjectWriteObject \
        (yTrainNumpyArray)

In [171]:
log_function \
    .DebugReturnObjectWriteObject \
        (yTestNumpyArray)

### **Create a StandardScaler Instance**

In [172]:
currentStandardScalar \
    = StandardScaler()

log_function \
    .DebugReturnObjectWriteObject \
        (currentStandardScalar)

### **Fit the StandardScaler**

In [173]:
XStandardScalar \
    = currentStandardScalar \
        .fit \
            (XTrainNumpyArray)

log_function \
    .DebugReturnObjectWriteObject \
        (XStandardScalar)

### **Scale the Data**

In [174]:
XTrainScaledNumpyArray \
    = XStandardScalar \
        .transform \
            (XTrainNumpyArray)

log_function \
    .DebugReturnObjectWriteObject \
        (XTrainScaledNumpyArray)

In [175]:
XTestScaledNumpyArray \
    = XStandardScalar \
        .transform \
            (XTestNumpyArray)

log_function \
    .DebugReturnObjectWriteObject \
        (XTestScaledNumpyArray)

# <br> **Section 3: Compile, Train, Evaluate, and Export the Model**

## **3.1: Compile Model**

### **Model Definition**

In [176]:
# These line of code define the model with the number of input features and hidden nodes for each layer.
neuralNetSequential \
    = tf.keras.models.Sequential()

log_function \
    .DebugReturnObjectWriteObject \
        (neuralNetSequential)

### **Layers**

In [177]:
inputDimensionInteger \
    = len(XTrainScaledNumpyArray[0])

# This line of code is the first layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 35,
              activation = 'relu',
              input_dim = inputDimensionInteger))

# This line of code is the hidden layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 54,
              activation = 'relu'))

# This line of code is the hidden layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 10,
              activation = 'relu'))

# This line of code is the hidden layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 91,
              activation = 'relu'))

# This line of code is the hidden layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 39,
              activation = 'relu'))

# This line of code is the output layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 1,
              activation = 'sigmoid'))

### **Check Model**

In [178]:
neuralNetSequential.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 35)                29400     
                                                                 
 dense_13 (Dense)            (None, 54)                1944      
                                                                 
 dense_14 (Dense)            (None, 10)                550       
                                                                 
 dense_15 (Dense)            (None, 91)                1001      
                                                                 
 dense_16 (Dense)            (None, 39)                3588      
                                                                 
 dense_17 (Dense)            (None, 1)                 40        
                                                                 
Total params: 36,523
Trainable params: 36,523
Non-trai

### **Compile**

In [179]:
neuralNetSequential \
    .compile \
        (loss = 'binary_crossentropy',
         optimizer = 'adam',
         metrics = ['accuracy'])

## **3.2: Train Model**

In [180]:
neuralNetSequential \
    .fit \
        (XTrainScaledNumpyArray,
         yTrainNumpyArray,
         epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.4823 - accuracy: 0.7732
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4198 - accuracy: 0.8029
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4132 - accuracy: 0.8041
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4081 - accuracy: 0.8067
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4057 - accuracy: 0.8083
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4043 - accuracy: 0.8085
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4026 - accuracy: 0.8088
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4024 - accuracy: 0.8104
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4008 - accuracy: 0.8097
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4013 - accura

## **3.3: Evaluate Model**

In [181]:
modelLossFloat, \
modelAccuracyFloat \
    = neuralNetSequential.evaluate \
        (XTestScaledNumpyArray,
         yTestNumpyArray,
         verbose = 2)

log_subroutine \
    .PrintAndLogWriteText \
        (f'\nModel Loss: {round(modelLossFloat * 100, 2)}%, '
         + f'Model Accuracy: {round(modelAccuracyFloat*100, 2)}%')

268/268 - 0s - loss: 0.5053 - accuracy: 0.8006 - 428ms/epoch - 2ms/step

Model Loss: 50.53%, Model Accuracy: 80.06%


## **3.4: Export Model**

In [182]:
neuralNetSequential.save \
    (CONSTANT_OPTIMIZATION_FILE_PATH)

In [183]:
#log_subroutine \
#    .EndProgramExecution()